In [1]:
import sys
import sklearn
import lightgbm
import imblearn
import pymongo
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from numpy import array

import datetime
from datetime import datetime,timedelta

import scipy
from scipy import stats
from scipy.stats import skew,kurtosis,describe,variation

from sklearn import model_selection
from sklearn.model_selection import cross_validate
import lightgbm
import xgboost
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, KFold 
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,roc_auc_score,classification_report,make_scorer

from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler

from pymongo import MongoClient

import warnings 
warnings.filterwarnings(action='ignore')

print('python version:',sys.version)
print('pandas version:',pd.__version__)
print('numpy version:',np.__version__)
print('scipy version:', scipy.__version__)
print('pymongo version:', pymongo.__version__)
print('sklearn version:', sklearn.__version__)
print('xgboost version:', xgboost.__version__)
print('lightgbm version:', lightgbm.__version__)
#print('imblearn version:', imblearn.__version__)
print('\n')
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

python version: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
pandas version: 2.0.3
numpy version: 1.23.1
scipy version: 1.9.1
pymongo version: 4.3.3
sklearn version: 1.2.2
xgboost version: 1.7.4
lightgbm version: 3.3.5


Code Running Time: 2023-09-15  15:43:1443S


### 1.) 데이터 업로드 

In [2]:
def get_mongo_data(db_name,collection_name,condition = False):
    server_address = "private_company_server"
    client_update = MongoClient("mongodb://private_client_address/admin".format(server_address))
    db = client_update[db_name]
    col = db[collection_name]
    df = pd.DataFrame(list(col.find({})))
    return df
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:43:1443S


In [3]:
data = get_mongo_data('privateDB','productiondata') # 실시간 데이터 업로드 
data = data.head(12522) # 2023-07-20일자 기준 
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S"))

Code Running Time: 2023-09-15  15:44:1544S


In [4]:
def data_organize(data):
    data = data.replace(['수직사출기1'], '수직사출기30호기') # 사출기 명칭 정정 
    data = data.replace(r'^s*$', float('NaN'), regex = True) # 사출기 명칭 빈 값 결측값으로 대체  
    data['Machine_Name'] = data['Machine_Name'].fillna('수직사출기30호기') # 사출기 명칭 채우기 
    ids = data.nunique().values>1 # 획일값만 갖는 인덱스 
    data_drop_distinct = data.loc[:,ids] # 획일값 제거 
    Final = data_drop_distinct.dropna() # 결측값 제거 [ Target변수에 전부 몰림 ]
    Final = Final.drop(Final.filter(regex='SETTING').columns, axis=1) # Setting값 제거 [ 데이터의 99%가 변동 없음]
    Final = Final.sort_values(by='TimeStamp') # TimeStamp 순서로 데이터 정렬 
    Final = Final.reset_index(drop=True)
    return Final
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:1644S


In [5]:
Final = data_organize(data)
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S"))
Final.head()

Code Running Time: 2023-09-15  15:44:1644S


,_id,AcquisitedTime,Working_No,MECHCD,SABUN,ITEMNM,Shot_Number,SHOT_DATE,WORKDAETIME_PRE,TimeStamp,CYCLE_TIME_PRE,CYCLE_TIME_CURR,TEMPLETE_ROTATION_PRE,TEMPLETE_ROTATION_CURR,TEMPLETE_OUT_TIME_PRE,TEMPLETE_OUT_TIME_CURR,TEMPLETE_IN_TIME_PRE,TEMPLETE_IN_TIME_CURR,INJ_TIME_PRE,INJ_TIME_CURR,WEIGHTING_TIME_PRE,weighting_time_curr,FILLING_TIME,CHANGING_PRES,CHANGING_POSITION,CHANGING_SPD,MAX_INJ_PRES,AVG_INJ_PRES,MAX_INJ_SPD,AVG_HOLDING_PRES1,AVG_HOLDING_PRES2,CUSHION,AVG_BACK_PRES,WEIGHTING_PRES,HOPPER_TEMP,HYD_OIL_TEMP,HOPPER_PREHEAT,HYD_OIL_PREHEAT,BBL_MIN_TEMP1,BBL_MIN_TEMP2,BBL_MIN_TEMP3,BBL_MIN_TEMP4,BBL_MIN_TEMP5,BBL_MIN_TEMP6,HOPPER_MIN_TEMP,HYD_OIL_MIN_TEMP,BBL_MAX_TEMP1,BBL_MAX_TEMP2,BBL_MAX_TEMP3,BBL_MAX_TEMP4,BBL_MAX_TEMP5,BBL_MAX_TEMP6,HOPPER_MAX_TEMP,HYD_OIL_MAX_TEMP,MOLD_TEMP1,MOLD_TEMP2,MOLD_TEMP3,MOLD_TEMP4,MOLD_TEMP5,MOLD_TEMP6,MOLD_TEMP7,MOLD_TEMP8,MOLD_TEMP_PREHEAT1,MOLD_TEMP_PREHEAT2,MOLD_TEMP_PREHEAT3,MOLD_TEMP_PREHEAT4,MOLD_TEMP_PREHEAT5,MOLD_TEMP_PREHEAT6,MOLD_TEMP_PREHEAT7,MOLD_TEMP_PREHEAT8,MOLD_MIN_TEMP1,MOLD_MIN_TEMP2,MOLD_MIN_TEMP3,MOLD_MIN_TEMP4,MOLD_MIN_TEMP5,MOLD_MIN_TEMP6,MOLD_MIN_TEMP7,MOLD_MIN_TEMP8,MOLD_MAX_TEMP1,MOLD_MAX_TEMP2,MOLD_MAX_TEMP3,MOLD_MAX_TEMP4,MOLD_MAX_TEMP5,MOLD_MAX_TEMP6,MOLD_MAX_TEMP7,MOLD_MAX_TEMP8,SHAPE_DIST_CURR,INJECTOR_DIST_CURR,INJ_DIST_CURR,NOZZLE_DIST_CURR,SHAPE_PRES_CURR,INJ_PRES_CURR,TEMPLETE_IN_LOW_SPD_2,TEMPLETE_IN_HIGH_SPD,TEMPLETE_IN_LOW_SPD_1,TEMPLETE_OUT_LOW_SPD_1,TEMPLETE_OUT_HIGH_SPD,TEMPLETE_OUT_LOW_SPD_2,TEMPLETE_OUT_HIGH_PRES_SPD,TEMPLETE_OUT_HIGH_SPD_PRES,TEMPLETE_OUT_LOW_SPD_PRES,TEMPLETE_OUT_HIGH_PRES,TEMPLETE_IN_HIGH_SPD_PRES,TEMPLETE_IN_COMP_PRES,TEMPLETE_IN_COMP_DIST,TEMPLETE_IN_LOW_SPD_DIST2,TEMPLETE_IN_HIGH_SPD_DIST,TEMPLETE_IN_LOW_SPD_DIST1,TEMPLETE_OUT_LOW_SPD_DIST1,TEMPLETE_OUT_HIGH_SPD_DIST,TEMPLETE_OUT_LOW_SPD_DIST2,EXT_FORWARD_COMP_DIST,EXT_FORWARD_HIGH_SPD_DIST,EXT_FORWARD_COMP_SPD,EXT_FORWARD_HIGH_SPD,EXT_FORWARD_PRES,EXT_BACK_COMP_DIST,EXT_BACK_HIGH_SPD_DIST,EXT_BACK_COMP_SPD,EXT_BACK_HIGH_SPD,EXT_BACK_PRES,NOZZLE_FORWARD_COMP_SPD,NOZZLE_FORWARD_HIGH_SPD,NOZZLE_FORWARD_COMP_DIST,NOZZLE_FORWARD_HIGH_SPD_DIST,NOZZLE_BACK_HIGH_SPD,NOZZLE_BACK_COMP_SPD,NOZZLE_BACK_HIGH_SPD_DIST,passorfail
0,63994e281574ea85c8f80eda,2022-12-14 04:16:40.122,20221214_00056,SJSCG-30,6Mv29-10000S,MV VALUE CASE,56,2022-12-13 15:00:00,8122,2022-12-14 04:16:20.202,7579,2497,610,610,784,780,563,565,630,630,535,427,189,829.0,210.0,305.0,828.0,547.0,605.0,609.0,644.0,179,40.0,80.0,67.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,350.0,350.0,350.0,295.0,60.0,100.0,60.0,148.0,152.0,149.0,26.0,26.0,9999.0,9999.0,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,579.0,607.0,0.0,115.0,40.0,15.0,60.0,10.0,15.0,80.0,20.0,60.0,60.0,30.0,78.0,80.0,80.0,3700.0,3500.0,3300.0,600.0,0.0,500.0,300.0,1300.0,700.0,13.0,27.0,30.0,600.0,1200.0,15.0,3.0,10.0,20.0,30.0,2.0,50.0,30.0,20.0,90.0,0.0
1,639958961574ea85c8f80f52,2022-12-14 05:01:10.132,20221214_00086,SJSCG-30,6Mv29-10000S,MV VALUE CASE,86,2022-12-13 15:00:00,8215,2022-12-14 05:00:50.165,7578,2223,610,611,782,782,566,567,630,631,515,149,189,838.0,210.0,293.0,837.0,539.0,611.0,621.0,645.0,181,40.0,80.0,70.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,350.0,350.0,350.0,295.0,60.0,100.0,60.0,148.0,151.0,149.0,27.0,28.0,9999.0,9999.0,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,578.0,183.0,0.0,116.0,649.0,15.0,60.0,10.0,15.0,80.0,20.0,60.0,60.0,30.0,78.0,80.0,80.0,3700.0,3500.0,3300.0,600.0,0.0,500.0,300.0,1300.0,700.0,13.0,27.0,30.0,600.0,1200.0,15.0,3.0,10.0,20.0,30.0,2.0,50.0,30.0,20.0,90.0,0.0
2,639971c81574ea85c8f81056,2022-12-14 06:48:40.123,20221214_00151,SJSCG-30,6Mv29-10000S,MV VALUE CASE,151,2022-12-13 15:00:00,8723,2022-12-14 06:48:10.151,7577,2077,611,611,778,780,567,566,631,630,539,5,187,800.0,210.0,298.0,800.0,515.0,604.0,588.0,645.0,180,40.0,80.0,70.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [6]:
Data = Final # 전체 데이터 
Target = 'passorfail' # 예측변수 Y 
Input_Data = Data.select_dtypes(include=[int,float]).drop(['Shot_Number',Target],axis=1) # 입력변수 X 
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:1744S


In [7]:
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))
Input_Data.head()

Code Running Time: 2023-09-15  15:44:1744S


,WORKDAETIME_PRE,CYCLE_TIME_PRE,CYCLE_TIME_CURR,TEMPLETE_ROTATION_PRE,TEMPLETE_ROTATION_CURR,TEMPLETE_OUT_TIME_PRE,TEMPLETE_OUT_TIME_CURR,TEMPLETE_IN_TIME_PRE,TEMPLETE_IN_TIME_CURR,INJ_TIME_PRE,INJ_TIME_CURR,WEIGHTING_TIME_PRE,weighting_time_curr,FILLING_TIME,CHANGING_PRES,CHANGING_POSITION,CHANGING_SPD,MAX_INJ_PRES,AVG_INJ_PRES,MAX_INJ_SPD,AVG_HOLDING_PRES1,AVG_HOLDING_PRES2,CUSHION,AVG_BACK_PRES,WEIGHTING_PRES,HOPPER_TEMP,HYD_OIL_TEMP,HOPPER_PREHEAT,HYD_OIL_PREHEAT,BBL_MIN_TEMP1,BBL_MIN_TEMP2,BBL_MIN_TEMP3,BBL_MIN_TEMP4,BBL_MIN_TEMP5,BBL_MIN_TEMP6,HOPPER_MIN_TEMP,HYD_OIL_MIN_TEMP,BBL_MAX_TEMP1,BBL_MAX_TEMP2,BBL_MAX_TEMP3,BBL_MAX_TEMP4,BBL_MAX_TEMP5,BBL_MAX_TEMP6,HOPPER_MAX_TEMP,HYD_OIL_MAX_TEMP,MOLD_TEMP1,MOLD_TEMP2,MOLD_TEMP3,MOLD_TEMP4,MOLD_TEMP5,MOLD_TEMP6,MOLD_TEMP7,MOLD_TEMP8,MOLD_TEMP_PREHEAT1,MOLD_TEMP_PREHEAT2,MOLD_TEMP_PREHEAT3,MOLD_TEMP_PREHEAT4,MOLD_TEMP_PREHEAT5,MOLD_TEMP_PREHEAT6,MOLD_TEMP_PREHEAT7,MOLD_TEMP_PREHEAT8,MOLD_MIN_TEMP1,MOLD_MIN_TEMP2,MOLD_MIN_TEMP3,MOLD_MIN_TEMP4,MOLD_MIN_TEMP5,MOLD_MIN_TEMP6,MOLD_MIN_TEMP7,MOLD_MIN_TEMP8,MOLD_MAX_TEMP1,MOLD_MAX_TEMP2,MOLD_MAX_TEMP3,MOLD_MAX_TEMP4,MOLD_MAX_TEMP5,MOLD_MAX_TEMP6,MOLD_MAX_TEMP7,MOLD_MAX_TEMP8,SHAPE_DIST_CURR,INJECTOR_DIST_CURR,INJ_DIST_CURR,NOZZLE_DIST_CURR,SHAPE_PRES_CURR,INJ_PRES_CURR,TEMPLETE_IN_LOW_SPD_2,TEMPLETE_IN_HIGH_SPD,TEMPLETE_IN_LOW_SPD_1,TEMPLETE_OUT_LOW_SPD_1,TEMPLETE_OUT_HIGH_SPD,TEMPLETE_OUT_LOW_SPD_2,TEMPLETE_OUT_HIGH_PRES_SPD,TEMPLETE_OUT_HIGH_SPD_PRES,TEMPLETE_OUT_LOW_SPD_PRES,TEMPLETE_OUT_HIGH_PRES,TEMPLETE_IN_HIGH_SPD_PRES,TEMPLETE_IN_COMP_PRES,TEMPLETE_IN_COMP_DIST,TEMPLETE_IN_LOW_SPD_DIST2,TEMPLETE_IN_HIGH_SPD_DIST,TEMPLETE_IN_LOW_SPD_DIST1,TEMPLETE_OUT_LOW_SPD_DIST1,TEMPLETE_OUT_HIGH_SPD_DIST,TEMPLETE_OUT_LOW_SPD_DIST2,EXT_FORWARD_COMP_DIST,EXT_FORWARD_HIGH_SPD_DIST,EXT_FORWARD_COMP_SPD,EXT_FORWARD_HIGH_SPD,EXT_FORWARD_PRES,EXT_BACK_COMP_DIST,EXT_BACK_HIGH_SPD_DIST,EXT_BACK_COMP_SPD,EXT_BACK_HIGH_SPD,EXT_BACK_PRES,NOZZLE_FORWARD_COMP_SPD,NOZZLE_FORWARD_HIGH_SPD,NOZZLE_FORWARD_COMP_DIST,NOZZLE_FORWARD_HIGH_SPD_DIST,NOZZLE_BACK_HIGH_SPD,NOZZLE_BACK_COMP_SPD,NOZZLE_BACK_HIGH_SPD_DIST
0,8122,7579,2497,610,610,784,780,563,565,630,630,535,427,189,829.0,210.0,305.0,828.0,547.0,605.0,609.0,644.0,179,40.0,80.0,67.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,350.0,350.0,350.0,295.0,60.0,100.0,60.0,148.0,152.0,149.0,26.0,26.0,9999.0,9999.0,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,579.0,607.0,0.0,115.0,40.0,15.0,60.0,10.0,15.0,80.0,20.0,60.0,60.0,30.0,78.0,80.0,80.0,3700.0,3500.0,3300.0,600.0,0.0,500.0,300.0,1300.0,700.0,13.0,27.0,30.0,600.0,1200.0,15.0,3.0,10.0,20.0,30.0,2.0,50.0,30.0,20.0,90.0
1,8215,7578,2223,610,611,782,782,566,567,630,631,515,149,189,838.0,210.0,293.0,837.0,539.0,611.0,621.0,645.0,181,40.0,80.0,70.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,350.0,350.0,350.0,295.0,60.0,100.0,60.0,148.0,151.0,149.0,27.0,28.0,9999.0,9999.0,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,578.0,183.0,0.0,116.0,649.0,15.0,60.0,10.0,15.0,80.0,20.0,60.0,60.0,30.0,78.0,80.0,80.0,3700.0,3500.0,3300.0,600.0,0.0,500.0,300.0,1300.0,700.0,13.0,27.0,30.0,600.0,1200.0,15.0,3.0,10.0,20.0,30.0,2.0,50.0,30.0,20.0,90.0
2,8723,7577,2077,611,611,778,780,567,566,631,630,539,5,187,800.0,210.0,298.0,800.0,515.0,604.0,588.0,645.0,180,40.0,80.0,70.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,350.0,350.0,350.0,295.0,60.0,100.0,60.0,151.0,150.0,153.0,23.0,25.0,9999.0,9999.0,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281.0,892.0,208.0,0.0,0.0,161.0,15.0,60.0,10.0,15.0,80.0,20.0,60.0,60.0,30.0,78.0,80.0,80.0,3700.0,3500.0,3300.0,600.0,0.0,500.0,300.0,1300.0,700.0,13.0,27.0,30.0,600.0,1200.0,15.0,3.0,10.0,20.0,30.0,2.0,50.0,30.0,20.0,90.0
3,8181,7579,2094,608,610,781,780,568,569,631,630,512,23,191,845.0,210.0,286.0,843.0,541.0,598.0,633.0,645.0,182,40.0,80.0,7

In [8]:
def describex(data):
    data = pd.DataFrame(data)
    stats = data.describe()
    skewness = data.skew()
    kurtosis = data.kurtosis()
    skewness_df = pd.DataFrame({'skewness':skewness}).T
    kurtosis_df = pd.DataFrame({'kurtosis':kurtosis}).T
    Appended_Stats = pd.concat([stats,skewness_df,kurtosis_df],axis=0)
    return Appended_Stats
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:1744S


In [9]:
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))
describex(Input_Data)

Code Running Time: 2023-09-15  15:44:1844S


,WORKDAETIME_PRE,CYCLE_TIME_PRE,CYCLE_TIME_CURR,TEMPLETE_ROTATION_PRE,TEMPLETE_ROTATION_CURR,TEMPLETE_OUT_TIME_PRE,TEMPLETE_OUT_TIME_CURR,TEMPLETE_IN_TIME_PRE,TEMPLETE_IN_TIME_CURR,INJ_TIME_PRE,INJ_TIME_CURR,WEIGHTING_TIME_PRE,weighting_time_curr,FILLING_TIME,CHANGING_PRES,CHANGING_POSITION,CHANGING_SPD,MAX_INJ_PRES,AVG_INJ_PRES,MAX_INJ_SPD,AVG_HOLDING_PRES1,AVG_HOLDING_PRES2,CUSHION,AVG_BACK_PRES,WEIGHTING_PRES,HOPPER_TEMP,HYD_OIL_TEMP,HOPPER_PREHEAT,HYD_OIL_PREHEAT,BBL_MIN_TEMP1,BBL_MIN_TEMP2,BBL_MIN_TEMP3,BBL_MIN_TEMP4,BBL_MIN_TEMP5,BBL_MIN_TEMP6,HOPPER_MIN_TEMP,HYD_OIL_MIN_TEMP,BBL_MAX_TEMP1,BBL_MAX_TEMP2,BBL_MAX_TEMP3,BBL_MAX_TEMP4,BBL_MAX_TEMP5,BBL_MAX_TEMP6,HOPPER_MAX_TEMP,HYD_OIL_MAX_TEMP,MOLD_TEMP1,MOLD_TEMP2,MOLD_TEMP3,MOLD_TEMP4,MOLD_TEMP5,MOLD_TEMP6,MOLD_TEMP7,MOLD_TEMP8,MOLD_TEMP_PREHEAT1,MOLD_TEMP_PREHEAT2,MOLD_TEMP_PREHEAT3,MOLD_TEMP_PREHEAT4,MOLD_TEMP_PREHEAT5,MOLD_TEMP_PREHEAT6,MOLD_TEMP_PREHEAT7,MOLD_TEMP_PREHEAT8,MOLD_MIN_TEMP1,MOLD_MIN_TEMP2,MOLD_MIN_TEMP3,MOLD_MIN_TEMP4,MOLD_MIN_TEMP5,MOLD_MIN_TEMP6,MOLD_MIN_TEMP7,MOLD_MIN_TEMP8,MOLD_MAX_TEMP1,MOLD_MAX_TEMP2,MOLD_MAX_TEMP3,MOLD_MAX_TEMP4,MOLD_MAX_TEMP5,MOLD_MAX_TEMP6,MOLD_MAX_TEMP7,MOLD_MAX_TEMP8,SHAPE_DIST_CURR,INJECTOR_DIST_CURR,INJ_DIST_CURR,NOZZLE_DIST_CURR,SHAPE_PRES_CURR,INJ_PRES_CURR,TEMPLETE_IN_LOW_SPD_2,TEMPLETE_IN_HIGH_SPD,TEMPLETE_IN_LOW_SPD_1,TEMPLETE_OUT_LOW_SPD_1,TEMPLETE_OUT_HIGH_SPD,TEMPLETE_OUT_LOW_SPD_2,TEMPLETE_OUT_HIGH_PRES_SPD,TEMPLETE_OUT_HIGH_SPD_PRES,TEMPLETE_OUT_LOW_SPD_PRES,TEMPLETE_OUT_HIGH_PRES,TEMPLETE_IN_HIGH_SPD_PRES,TEMPLETE_IN_COMP_PRES,TEMPLETE_IN_COMP_DIST,TEMPLETE_IN_LOW_SPD_DIST2,TEMPLETE_IN_HIGH_SPD_DIST,TEMPLETE_IN_LOW_SPD_DIST1,TEMPLETE_OUT_LOW_SPD_DIST1,TEMPLETE_OUT_HIGH_SPD_DIST,TEMPLETE_OUT_LOW_SPD_DIST2,EXT_FORWARD_COMP_DIST,EXT_FORWARD_HIGH_SPD_DIST,EXT_FORWARD_COMP_SPD,EXT_FORWARD_HIGH_SPD,EXT_FORWARD_PRES,EXT_BACK_COMP_DIST,EXT_BACK_HIGH_SPD_DIST,EXT_BACK_COMP_SPD,EXT_BACK_HIGH_SPD,EXT_BACK_PRES,NOZZLE_FORWARD_COMP_SPD,NOZZLE_FORWARD_HIGH_SPD,NOZZLE_FORWARD_COMP_DIST,NOZZLE_FORWARD_HIGH_SPD_DIST,NOZZLE_BACK_HIGH_SPD,NOZZLE_BACK_COMP_SPD,NOZZLE_BACK_HIGH_SPD_DIST
count,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.0,329.000000,329.000000,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.000000,329.0,329.000000,329.000000,329.000000,329.000000,329.000000,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.000000,329.0,329.0,329.0,329.0,329.000000,329.0,329.0,329.0,329.0,329.000000,329.0,329.0,329.0,329.000000,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0,329.0
mean,8594.820669,7487.349544,2126.173252,608.027356,608.127660,772.000000,775.969605,570.723404,573.832827,625.252280,625.966565,551.121581,271.723404,187.799392,773.255319,208.024316,298.735562,771.750760,497.547112,561.115502,555.121581,616.802432,172.407295,39.957447,80.0,74.474164,39.109422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,350.0,350.0,350.0,295.0,60.0,130.699088,60.0,154.431611,147.088146,148.665653,37.085106,37.103343,9999.0,9999.0,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.753799,581.993921,243.966565,0.620061,97.878419,149.191489,15.197568,60.0,10.0,15.0,80.0,20.0,60.0,60.0,30.0,65.936170,80.0,80.0,3700.0,3500.0,3296.048632,600.0,0.0,500.0,300.0,1296.048632,700.0,13.0,27.0,30.395137,600.0,1200.0,15.0,3.0,10.0,20.0,30.0,2.0,50.0,30.0,20.0,90.0
std,2231.981757,542.128233,188.581071,1.421475,1.419204,55.429421,35.300439,55.520976,37.661919,76.115446,70.5844

### 2.) Train/Test 데이터 분할 

In [10]:
def Train_Test_Setting(Data,Input_Data,test_size):
    X = Input_Data
    y = Data[Target]
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size,random_state=2021,stratify=y)
    ros_test = RandomUnderSampler(sampling_strategy=0.7,random_state=2021)   
    X_test,y_test = ros_test.fit_resample(X_test,y_test)    
    return X_train, y_train, X_test, y_test
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:1844S


### 3.) 입력변수 선택 

In [11]:
def T_Test_Feature_Selection(Data,Input_Data,test_size,Level):
    X = Input_Data
    y = Data[Target]
    X_train, y_train, X_test, y_test = Train_Test_Setting(Data,Input_Data,test_size)

    # Train Data 기준 변수 선택 
    Train = pd.concat([X_train,y_train],axis=1)
    Pass = Train[Train['passorfail']==0].drop(['passorfail'],axis=1)
    Fail = Train[Train['passorfail']==1].drop(['passorfail'],axis=1)

    # Welch's T-Test 기준 변수 선택 
    T_Test = stats.ttest_ind(Pass,Fail,equal_var=False) 
    Columns = pd.DataFrame(X_train.columns).rename(columns={0:'Columns'})
    P_Value = pd.DataFrame(T_Test[1]).rename(columns={0:'P-Value'})
    Concat_columns_pvalues = pd.concat([Columns,P_Value],axis=1).dropna()
    
    Significant_Values = Concat_columns_pvalues[Concat_columns_pvalues['P-Value'] < Level]['Columns'].to_list()
    T_Test_Selected = Data[Significant_Values]
    return T_Test_Selected
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:1844S


In [12]:
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))
df_ttest = T_Test_Feature_Selection(Data,Input_Data,test_size=0.6,Level = 0.05) # 0.01, 0.05, 0.1

print('T-Test 기반 변수선택 : 119  --> ',df_ttest.shape[1])
print('\n')
print('입력변수 목록:\n',df_ttest.columns) 

Code Running Time: 2023-09-15  15:44:1944S
T-Test 기반 변수선택 : 119  -->  33


입력변수 목록:
 Index(['WORKDAETIME_PRE', 'CYCLE_TIME_PRE', 'CYCLE_TIME_CURR',
       'TEMPLETE_OUT_TIME_PRE', 'TEMPLETE_OUT_TIME_CURR',
       'TEMPLETE_IN_TIME_CURR', 'INJ_TIME_PRE', 'INJ_TIME_CURR',
       'WEIGHTING_TIME_PRE', 'FILLING_TIME', 'CHANGING_PRES',
       'CHANGING_POSITION', 'MAX_INJ_PRES', 'AVG_INJ_PRES',
       'AVG_HOLDING_PRES1', 'AVG_HOLDING_PRES2', 'CUSHION', 'AVG_BACK_PRES',
       'HOPPER_TEMP', 'HYD_OIL_TEMP', 'HOPPER_MAX_TEMP', 'MOLD_TEMP1',
       'MOLD_TEMP2', 'MOLD_TEMP3', 'MOLD_TEMP4', 'MOLD_TEMP5', 'INJ_DIST_CURR',
       'SHAPE_PRES_CURR', 'TEMPLETE_IN_LOW_SPD_2', 'TEMPLETE_OUT_HIGH_PRES',
       'TEMPLETE_IN_HIGH_SPD_DIST', 'EXT_FORWARD_COMP_DIST',
       'EXT_FORWARD_PRES'],
      dtype='object')


In [13]:
### 입력 변수 선택 이후 데이터 재분할 
X_train, y_train, X_test, y_test = Train_Test_Setting(Data,df_ttest,test_size = 0.6) 
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:2044S


### 4.) 예측 모델 선택 

In [14]:
# 활용할 ML 나열 
rf = RandomForestClassifier(random_state=2021)
rf_bal = RandomForestClassifier(random_state=2021,class_weight='balanced')
ad = AdaBoostClassifier(random_state=2021)
gbm = GradientBoostingClassifier(random_state=2021)
xgb = XGBClassifier(random_state=2021)
xgb_bal = XGBClassifier(random_state=2021,scale_pos_weight=y_train.value_counts()[0]/y_train.value_counts()[1])
lgbm =  LGBMClassifier(random_state=2021)
lgbm_bal = LGBMClassifier(random_state=2021,scale_pos_weight=y_train.value_counts()[0]/y_train.value_counts()[1])
models = [rf,rf_bal,ad,gbm,xgb,xgb_bal,lgbm,lgbm_bal]
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:2344S


#### 4-1.) 일반 예측 

In [15]:
def get_clf_eval(y_test,pred=None,pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred_proba)
    f1 = f1_score(y_test,pred)
    print('정확도:',accuracy.round(3),'정밀도:', precision.round(3),'재현율:',recall.round(3),'AUC:',roc_auc.round(3),'F1:',f1.round(3))
    
def test_model(model):
    ml = model
    ml.fit(X_train,y_train)      
    pred = ml.predict(X_test)
    pred_proba = ml.predict_proba(X_test)[:,1]
    get_clf_eval(y_test,pred,pred_proba)    
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:2444S


In [16]:
test_model(lgbm_bal)

print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

정확도: 0.952 정밀도: 0.985 재현율: 0.885 AUC: 0.932 F1: 0.933
Code Running Time: 2023-09-15  15:44:2644S


#### 4-2.) 교차검증 

In [17]:
def Train_Test_MultiSplit(Data,Input_Data,test_size,i):
    ros_test = RandomUnderSampler(sampling_strategy=0.7,random_state=2021)
    X = Input_Data
    y = Data[Target]
    X_train_No_i,X_test_No_i,y_train_No_i,y_test_No_i = train_test_split(X,y,test_size=test_size,shuffle=True,stratify=y)   
    X_test_No_i,y_test_No_i = ros_test.fit_resample(X_test_No_i,y_test_No_i)  
    return X_train_No_i,X_test_No_i,y_train_No_i,y_test_No_i
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:2644S


In [18]:
#### 기존 전체 변수를 가진 데이터셋으로 5회 랜덤 분할
X_train_No1,X_test_No1,y_train_No1,y_test_No1 = Train_Test_MultiSplit(Data,Input_Data,test_size=0.6,i=1)
X_train_No2,X_test_No2,y_train_No2,y_test_No2 = Train_Test_MultiSplit(Data,Input_Data,test_size=0.6,i=2)
X_train_No3,X_test_No3,y_train_No3,y_test_No3 = Train_Test_MultiSplit(Data,Input_Data,test_size=0.6,i=3)
X_train_No4,X_test_No4,y_train_No4,y_test_No4 = Train_Test_MultiSplit(Data,Input_Data,test_size=0.6,i=4)
X_train_No5,X_test_No5,y_train_No5,y_test_No5 = Train_Test_MultiSplit(Data,Input_Data,test_size=0.6,i=5)
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:2844S


In [19]:
def CV_T_Test_Feature_Selection(X_train,y_train,X_test):
    Train = pd.concat([X_train,y_train],axis=1)
    Pass = Train[Train['passorfail']==0].drop(['passorfail'],axis=1)
    Fail = Train[Train['passorfail']==1].drop(['passorfail'],axis=1)
    T_Test = stats.ttest_ind(Pass,Fail,equal_var=False) # equal_var=True
    Columns = pd.DataFrame(X_train.columns).rename(columns={0:'Columns'})
    P_Value = pd.DataFrame(T_Test[1]).rename(columns={0:'P-Value'})
    Concat_columns_pvalues = pd.concat([Columns,P_Value],axis=1).dropna()
    Significant_Values = Concat_columns_pvalues[Concat_columns_pvalues['P-Value'] < 0.05]['Columns'].to_list()
    df_ttest = Data[Significant_Values]
    if len(df_ttest.columns) == 0: 
        X_train = X_train[Input_Data.columns] 
        X_test = X_test[Input_Data.columns]
        return X_train,X_test 
    else :
        X_train = X_train[df_ttest.columns]
        X_test = X_test[df_ttest.columns]
        return X_train,X_test
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:2944S


In [20]:
#### Welch's T-Test 기준 변수 선택 & 5회 랜덤 분할 
X_train_No1,X_test_No1 = CV_T_Test_Feature_Selection(X_train_No1,y_train_No1,X_test_No1)
X_train_No2,X_test_No2 = CV_T_Test_Feature_Selection(X_train_No2,y_train_No2,X_test_No2)
X_train_No3,X_test_No3 = CV_T_Test_Feature_Selection(X_train_No3,y_train_No3,X_test_No3)
X_train_No4,X_test_No4 = CV_T_Test_Feature_Selection(X_train_No4,y_train_No4,X_test_No4)
X_train_No5,X_test_No5 = CV_T_Test_Feature_Selection(X_train_No5,y_train_No5,X_test_No5)
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:3144S


In [21]:
def crossvalidate_model(model):
    ml = model
    ml.fit(X_train_No1,y_train_No1)      
    pred_No1 = ml.predict(X_test_No1)
    Fold1 = pd.DataFrame({'Accuracy':[accuracy_score(y_test_No1,pred_No1).round(3)],
                      'Precision':[precision_score(y_test_No1,pred_No1).round(3)],
                      'Recall':[recall_score(y_test_No1,pred_No1).round(3)],
                      'F1_score':[f1_score(y_test_No1,pred_No1).round(3)],
                      'AUC_ROC_score':[roc_auc_score(y_test_No1,pred_No1).round(3)]},
                       index = ['Test1'])
    ml.fit(X_train_No2,y_train_No2)      
    pred_No2 = ml.predict(X_test_No2)
    Fold2 = pd.DataFrame({'Accuracy':[accuracy_score(y_test_No2,pred_No2).round(3)],
                      'Precision':[precision_score(y_test_No2,pred_No2).round(3)],
                      'Recall':[recall_score(y_test_No2,pred_No2).round(3)],
                      'F1_score':[f1_score(y_test_No2,pred_No2).round(3)],
                      'AUC_ROC_score':[roc_auc_score(y_test_No2,pred_No2).round(3)]},
                       index = ['Test2'])
    ml.fit(X_train_No3,y_train_No3)      
    pred_No3 = ml.predict(X_test_No3)
    Fold3 = pd.DataFrame({'Accuracy':[accuracy_score(y_test_No3,pred_No3).round(3)],
                      'Precision':[precision_score(y_test_No3,pred_No3).round(3)],
                      'Recall':[recall_score(y_test_No3,pred_No3).round(3)],
                      'F1_score':[f1_score(y_test_No3,pred_No3).round(3)],
                      'AUC_ROC_score':[roc_auc_score(y_test_No3,pred_No3).round(3)]},
                       index = ['Test3'])
    ml.fit(X_train_No4,y_train_No4)      
    pred_No4 = ml.predict(X_test_No4)
    Fold4 = pd.DataFrame({'Accuracy':[accuracy_score(y_test_No4,pred_No4).round(3)],
                      'Precision':[precision_score(y_test_No4,pred_No4).round(3)],
                      'Recall':[recall_score(y_test_No4,pred_No4).round(3)],
                      'F1_score':[f1_score(y_test_No4,pred_No4).round(3)],
                      'AUC_ROC_score':[roc_auc_score(y_test_No4,pred_No4).round(3)]},
                       index = ['Test4'])
    ml.fit(X_train_No5,y_train_No5)      
    pred_No5 = ml.predict(X_test_No5)
    Fold5 = pd.DataFrame({'Accuracy':[accuracy_score(y_test_No5,pred_No5).round(3)],
                      'Precision':[precision_score(y_test_No5,pred_No5).round(3)],
                      'Recall':[recall_score(y_test_No5,pred_No5).round(3)],
                      'F1_score':[f1_score(y_test_No5,pred_No5).round(3)],
                      'AUC_ROC_score':[roc_auc_score(y_test_No5,pred_No5).round(3)]},
                       index = ['Test5'])
    Folds = pd.concat([Fold1,Fold2,Fold3,Fold4,Fold5],axis=0)
    Fold_Mean = pd.DataFrame({'Accuracy':[Folds['Accuracy'].mean().round(3)],
                      'Precision':[Folds['Precision'].mean().round(3)],
                      'Recall':[Folds['Recall'].mean().round(3)],
                      'F1_score':[Folds['F1_score'].mean().round(3)],
                      'AUC_ROC_score':[Folds['AUC_ROC_score'].mean().round(3)]},
                       index = ['Mean_Test_Score'])
    Run_Tests = pd.concat([Folds,Fold_Mean],axis=0)
    return Run_Tests

In [22]:
Run_Tests = crossvalidate_model(lgbm_bal)
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))

Code Running Time: 2023-09-15  15:44:3544S


In [23]:
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S"))
Run_Tests

Code Running Time: 2023-09-15  15:44:3644S


,Accuracy,Precision,Recall,F1_score,AUC_ROC_score
Test1,0.952,0.985,0.885,0.932,0.935
Test2,0.965,0.980,0.975,0.980,0.980
Test3,0.952,0.985,0.885,0.932,0.935
Test4,0.952,0.985,0.885,0.932,0.935
Test5,0.985,0.970,0.980,0.975,0.985
Mean_Test_Score,0.961,0.981,0.922,0.950,0.954


In [24]:
def TTA(model):
    model.fit(X_train,y_train) # 모델 정의 & 학습
    pred = model.predict(X_test) # 1회 예측  
    Run_Tests = crossvalidate_model(model) # 교차 검증 
    print('Mean F1-Score :', Run_Tests.iloc[-1][-2]) # F1-Score f1_score(y_test,pred).round(3)

In [25]:
print('Code Running Time:',datetime.today().strftime("%Y-%m-%d  %H:%M:%S%MS"))
TTA(lgbm_bal) # model 선택 

Code Running Time: 2023-09-15  15:44:3744S
Mean F1-Score : 0.95
